## 对phenos.xlsx文件数据分析

In [1]:
import pandas as pd

In [13]:
# 读取xlsx文件
file_path = '../../dataSource/phenos.xlsx'
df = pd.read_excel(file_path,header=4)
df.head()

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Treatment,Replicate,Entry,Block,Plot number,Field row,Field column,Days till flowering (50%) (DTF),Chlorophyll content index (CCI),Leaf area index (%) (LAI),"Flowering success (flowering and filled grain production occurred, yes or no)",Fecundity (Filled grain number)
0,Wet,1,e40,1,1,1,1,73.0,7.8,0.292157,1,180.0
1,Dry,1,e40,1,1,1,1,71.0,17.2,0.153312,1,16.0
2,Wet,1,e167,1,2,2,1,68.0,7.5,4.482489,1,1010.0
3,Dry,1,e167,1,2,2,1,71.0,7.3,0.256249,1,139.0
4,Wet,1,e115,1,3,3,1,57.0,8.9,4.415642,1,961.0


### 数据处理，排序，去除无用字段

In [14]:
#将Entry字段的e去掉，然后转换为int类型，再赋值给Entry字段
df['EntryNumber'] = df['Entry'].str[1:].astype(int)

#根据新的字段进行排序，排序规则如下，先按Replicate字段升序，再按Treatment字段升序，最后按EntryNumber字段升序
df = df.sort_values(by=['EntryNumber','Treatment','Replicate'])

#以Treatment字段，Entry字段，Replicate字段组合成一个新的字段，例如Dry_e1_1
df['Entry_Treatment_Replicate'] = df['Entry'].astype(str) + '_'+df['Treatment'] + '_'  + df['Replicate'].astype(str)
df['Entry_Treatment'] = df['Entry'].astype(str) + '_'+df['Treatment']
#将新的字段Entry_Treatment_Replicate放到第一列
first_col = 'Entry_Treatment_Replicate'
second_col = 'Entry_Treatment'
df = pd.concat([df[first_col],df[second_col], df.drop(columns=[first_col,second_col])], axis=1)


#删除无用字段
df = df.drop(columns=['EntryNumber','Treatment','Replicate','Block','Plot number','Field row','Field column'])

#到处到新的xlsx文件
df.to_excel('../results/phenos_analyze_sorted.xlsx',index=False)


### 数据分析

In [15]:
df = pd.read_excel('../results/phenos_analyze_sorted.xlsx')

#处理Days till flowering (50%) (DTF)，Leaf area index (%) (LAI)，Flowering success (flowering and filled grain production occurred, yes or no)，Fecundity (Filled grain number)字段
df['DTF_mean'] = df.groupby('Entry_Treatment')['Days till flowering (50%) (DTF)'].transform('mean')
df['DTF_std'] = df.groupby('Entry_Treatment')['Days till flowering (50%) (DTF)'].transform('std')
df['DTF_cv'] = df['DTF_std']/df['DTF_mean']
#新增一列，如果变异系数大于0.1,那么就是1，否则就是0
df['DTF_cv_flag'] = df['DTF_cv'].apply(lambda x: 1 if x>0.15 else 0)
#统计df['DTF_cv_flag']为1的个数
print(df['DTF_cv_flag'].sum())
#15
# (440-15/3)/440 = 0.99

#计算e1_Dry_1,e1_Dry_2e1_Dry_3的CCI的平均值，然后赋值给e1_Dry_1,e1_Dry_2e1_Dry_3的CCI_mean字段..计算e2_Dry_1,e2_Dry_2e2_Dry_3的CCI的平均值，然后赋值给e2_Dry_1,e2_Dry_2e2_Dry_3的CCI_mean
df['CCI_mean'] = df.groupby('Entry_Treatment')['Chlorophyll content index (CCI)'].transform('mean')
#计算标准差
df['CCI_std'] = df.groupby('Entry_Treatment')['Chlorophyll content index (CCI)'].transform('std')
#计算变异系数
df['CCI_cv'] = df['CCI_std']/df['CCI_mean']
#新增一列，如果变异系数大于0.1,那么就是1，否则就是0
df['CCI_cv_flag'] = df['CCI_cv'].apply(lambda x: 1 if x<=0.15 else 0)
#统计df['CCI_cv_flag']为1的个数
print(df['CCI_cv_flag'].sum())
#93
# (440-93/3)/440 = 0.92


df['LAI_mean'] = df.groupby('Entry_Treatment')['Leaf area index (%) (LAI)'].transform('mean')
df['LAI_std'] = df.groupby('Entry_Treatment')['Leaf area index (%) (LAI)'].transform('std')
df['LAI_cv'] = df['LAI_std']/df['LAI_mean']
df['LAI_cv_flag'] = df['LAI_cv'].apply(lambda x: 1 if x<=0.15 else 0)
#统计df['LAI_cv_flag']为1的个数,标注在LAI_cv_flag这一列的最后
print(df['LAI_cv_flag'].sum())
#72
# (440-72/3)/440 = 0.94

df['Flowering success_mean'] = df.groupby('Entry_Treatment')['Flowering success (flowering and filled grain production occurred, yes or no)'].transform('mean')
df['Flowering success_std'] = df.groupby('Entry_Treatment')['Flowering success (flowering and filled grain production occurred, yes or no)'].transform('std')
df['Flowering success_cv'] = df['Flowering success_std']/df['Flowering success_mean']
df['Flowering success_cv_flag'] = df['Flowering success_cv'].apply(lambda x: 1 if x<=0.15 else 0)
print(df['Flowering success_cv_flag'].sum())
#678
# (440-678/3)/440 = 0.49

df['Fecundity_mean'] = df.groupby('Entry_Treatment')['Fecundity (Filled grain number)'].transform('mean')
df['Fecundity_std'] = df.groupby('Entry_Treatment')['Fecundity (Filled grain number)'].transform('std')
df['Fecundity_cv'] = df['Fecundity_std']/df['Fecundity_mean']
df['Fecundity_cv_flag'] = df['Fecundity_cv'].apply(lambda x: 1 if x<=0.15 else 0)
print(df['Fecundity_cv_flag'].sum())
#678
# (440-87/3)/440 = 0.93

#将处理后的数据导出到新的xlsx文件
df.to_excel('../results/phenos_analyze_result.xlsx',index=False)

15
93
72
678
87
